## 오늘은
- MNIST에 CNN 적용해보기
    1. 딥러닝을 학습시키는 단계
    2. 우리가 만들 CNN 구조 확인
    3. MNIST에 CNN 적용 코드를 함께 작성
    
## 학습 단계(code 기준)
1. 라이브러리 가져오고 (torch, torchvision, matplotlib 같은 것들)
2. GPU 사용 설정하고 random value를 위한 seed 설정!
3. 학습에 사용되는 parameter 설정! (learning_rate, training_epochs, batch_size, etc)
4. 데이터셋을 가져오고 (학습에 쓰기 편하게) loader 만들기
5. 학습 모델 만들기 (class CNN(torch.nn.Module))
6. Loss function (Criterion)을 선택하고 최적화 도구 선택(optimizer)
7. 모델 학습 및 loss check(Criterion의 output)
8. 학습된 모델의 성능을 확인한다.


![snapshot](./10-2-0.png)

- (Layer 1) Convolution layer = (in_c=1, out_c=32, kernel_size=3, stride=1, padding=1)
- (Layer 1) MaxPool layer = (kernel_size=2, stride=2)
- (Layer 2) Convolution layer = (in_c=32, out_c=64, kernel_size=3, stride=1, padding=1)
- (Layer 2) MaxPool layer = (kernel_size=2, stride=2)
- view => (batch_size x [7,7,64] => batch_size x [3136])
- Fully_Connect layer => (input=3136, output=10)

view라는 건 사실 레이어로 존재하진 않지만 이해의 편의를 위해 넣어둠.

In [1]:
import torch
import torch.nn as nn
inputs = torch.Tensor(1, 1, 28, 28)   # batch, channel, height, width
inputs.shape

torch.Size([1, 1, 28, 28])

In [2]:
conv1 = nn.Conv2d(1, 32, 3, padding=1)   # 1채널 in, 32채널 out, 커널 사이즈 3, 패딩 1
conv2 = nn.Conv2d(32, 64, 3, padding=1)  # stride 기본값은 1이므로 지정할 필요 X
pool = nn.MaxPool2d(2)   # max pooling은 똑같은 역할 담당하므로 하나만 만들기

In [3]:
'''
                 (height - kernel) + 2 * padding       (28 - 3) + 2 * 1
    new_height = ------------------------------- + 1 = ---------------- + 1 = 28
                              stride                           1
'''

out = conv1(inputs)
out.shape

[W NNPACK.cpp:53] Could not initialize NNPACK! Reason: Unsupported hardware.


torch.Size([1, 32, 28, 28])

In [4]:
'''
                 height   28
    new_height = ------ = -- = 14
                 kernel    2
'''

out = pool(out)
out.shape

torch.Size([1, 32, 14, 14])

In [5]:
out = conv2(out)
out.shape

torch.Size([1, 64, 14, 14])

In [6]:
out = pool(out)
out.shape

torch.Size([1, 64, 7, 7])

In [7]:
# [1, 32, 14, 14]에서 각 위치에 해당하는 값 리턴하는 코드

print(out.size(0), out.size(1), out.size(2), out.size(3))

1 64 7 7


In [8]:
# 64 * 7 * 7 = 3136

out = out.view(out.size(0), -1)
out.shape

torch.Size([1, 3136])

In [9]:
# in이 3136, out이 10인 fully-connected layer 만들기

fc = nn.Linear(3136, 10)
out = fc(out)
out.shape

torch.Size([1, 10])

... 대충 이러한 방식


## MNIST에 CNN 적용 코드를 함께 작성
- jupyter notebook을 켜고 직접 같이 쳐보면서 진행하겠습니다.
- 그럼 시작해볼까요?

In [22]:
# 1. 초기 라이브러리 불러오기

import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import torch.nn.init

In [16]:
# 2. GPU 설정, random value를 위한 seed 설정

device = "cuda" if torch.cuda.is_available() else "cpu"

torch.manual_seed(777)    # random value 고정해주는 코드
if device == "cuda":
    torch.cuda.manual_seed_all(777)

In [25]:
print(device, torch.cuda.is_available())

cpu False


In [19]:
# 3. 학습에 사용하는 파라미터 설정
# 배포하는 코드에는 여기에 파라미터를 작성하는 걸로 되어 있는데, 사실 파라미터는 나중에 써도 상관없음.

learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [20]:
# 4. 데이터셋 가져오고,
# MNIST dataset

mnist_train = dsets.MNIST(root="MNIST_data/",
                         train=True,
                         transform=transforms.ToTensor(),
                         download=True)
# transform : input 데이터를 어떻게 변환해 줄거냐?
# 그냥 받아왔을 때는 tensor value가 아님. 그러니 MNIST data를 텐서 밸류로 변환해주는 역할
mnist_test = dsets.MNIST(root="MNIST_data/",
                        train=False,
                        transform=transforms.ToTensor(),
                        download=True)

In [21]:
# 4. (학습에 쓰기 편하게) 데이터 loader 만들기

data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                         batch_size=batch_size,
                                         shuffle=True,  # 데이터를 섞은 상태로 던져줘라
                                         drop_last=True)

In [27]:
# 5. CNN이라는 학습 모델 만들기

class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()  # 이거 절대 빼먹으면 안됨. 빼먹으면 학습이 전혀 안된다.
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc = nn.Linear(7*7*64, 10, bias=True)   # 0-9가 10개니까
        torch.nn.init.xavier_uniform_(self.fc.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        
        # view로 펼치기. 배치 사이즈만큼 펼치고, 나머지는 한 줄로.
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


In [29]:
model = CNN().to(device)

In [30]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)   # Adam optimizer 사용

In [33]:
# Training
total_batch = len(data_loader)

for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:   # (image, label)
        X = X.to(device)
        Y = Y.to(device)
        # cuda 연산을 진행하려면 torch.tensor가 아니라 torch.cuda_tensor가 되어 있어야 함
        
        optimizer.zero_grad()   # 이거 빼먹으면 학습이 아예 안됨
        hypothesis = model(X)  # model에다가 input 값 넣어서 그 출력값이 가설이 되겠지
        
        cost = criterion(hypothesis, Y)  # 가설과 실제 라벨 사이의 차이를 계산하는 loss. cross entropy로 계산을 한다.
        cost.backward()   # 그리고 그 cost 값을 backward를 해주고.
        optimizer.step()   # optimizer가 그 값을 토대로 학습을 진행함.
        
        avg_cost += cost / total_batch    # 그 다음에 average cost 계산해서 쌓아준다.
        
    # 한 epoch이 끝나면 epoch에 대해서 cost가 몇이었는지 확인을 하면 됨
    print("[Epoch:{}] cost = {}".format(epoch + 1, avg_cost))
print("Learning Finished!")

[Epoch:1] cost = 0.22519506514072418
[Epoch:2] cost = 0.059969183057546616
[Epoch:3] cost = 0.04360518231987953
[Epoch:4] cost = 0.03506407514214516
[Epoch:5] cost = 0.02890731394290924
[Epoch:6] cost = 0.024456340819597244
[Epoch:7] cost = 0.02169337309896946
[Epoch:8] cost = 0.017144329845905304
[Epoch:9] cost = 0.014846979640424252
[Epoch:10] cost = 0.012522861361503601
[Epoch:11] cost = 0.010015181265771389
[Epoch:12] cost = 0.010326125659048557
[Epoch:13] cost = 0.007664399221539497
[Epoch:14] cost = 0.006844808347523212
[Epoch:15] cost = 0.005508054979145527
Learning Finished!


In [35]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    # 한번에 집어넣으려고 쫙 펼쳤다고 생각하면 된다.
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print("Accuracy:", accuracy.item())

/Users/sanghoon/opt/anaconda3/lib/python3.9/site-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9883000254631042


## 우리가 확인한 결과
- 더 깊게 레이어를 쌓으면 어떻게 될까?
- 더 잘 되지 않을까? (위의 것은 98.5%)

- 더 추가하자! (Layer 3, Layer 4, 5) (NLL: negative log-likelihood)
![snapshot](./10-2-1.png)

In [41]:
class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()  # 이거 절대 빼먹으면 안됨. 빼먹으면 학습이 전혀 안된다.
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        ########
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        # 7을 maxpool(2)하면 3, 출력은 128
        self.fc1 = nn.Linear(3*3*128, 625)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(625, 10, bias=True)   # 0-9가 10개니까
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        ########
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        
        # view로 펼치기. 배치 사이즈만큼 펼치고, 나머지는 한 줄로.
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

model = CNN().to(device)

# model을 만들면 테스트를 해봐야 한다.
value = torch.Tensor(1, 1, 28, 28).to(device)
print( (model(value)).shape )

torch.Size([1, 10])


In [43]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)   # Adam optimizer 사용

# Training
total_batch = len(data_loader)

for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:   # (image, label)
        X = X.to(device)
        Y = Y.to(device)
        # cuda 연산을 진행하려면 torch.tensor가 아니라 torch.cuda_tensor가 되어 있어야 함
        
        optimizer.zero_grad()   # 이거 빼먹으면 학습이 아예 안됨
        hypothesis = model(X)  # model에다가 input 값 넣어서 그 출력값이 가설이 되겠지
        
        cost = criterion(hypothesis, Y)  # 가설과 실제 라벨 사이의 차이를 계산하는 loss. cross entropy로 계산을 한다.
        cost.backward()   # 그리고 그 cost 값을 backward를 해주고.
        optimizer.step()   # optimizer가 그 값을 토대로 학습을 진행함.
        
        avg_cost += cost / total_batch    # 그 다음에 average cost 계산해서 쌓아준다.
        
    # 한 epoch이 끝나면 epoch에 대해서 cost가 몇이었는지 확인을 하면 됨
    print("[Epoch:{}] cost = {}".format(epoch + 1, avg_cost))
print("Learning Finished!")

with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    # 한번에 집어넣으려고 쫙 펼쳤다고 생각하면 된다.
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print("Accuracy:", accuracy.item())

[Epoch:1] cost = 0.15921327471733093
[Epoch:2] cost = 0.04140569642186165
[Epoch:3] cost = 0.028637103736400604
[Epoch:4] cost = 0.021885210648179054
[Epoch:5] cost = 0.01738460548222065
[Epoch:6] cost = 0.014934803359210491
[Epoch:7] cost = 0.012485607527196407
[Epoch:8] cost = 0.011487196199595928
[Epoch:9] cost = 0.010386358015239239
[Epoch:10] cost = 0.0076933130621910095
[Epoch:11] cost = 0.005744565278291702
[Epoch:12] cost = 0.006951894611120224
[Epoch:13] cost = 0.004991516470909119
[Epoch:14] cost = 0.006859457585960627
[Epoch:15] cost = 0.005690425168722868
Learning Finished!
Accuracy: 0.9876999855041504


**오히려 아까보다 accuracy가 떨어짐!** (98% -> 97%)

모델을 쌓을 때는 깊게 쌓는 것도 중요하지만, 얼마나 효율적으로 쌓는지가 더 중요하다는 것도 생각하면서 쌓아보면 좋을 것이다.

이후에 배울 굉장히 다양한 아키텍처들. 언급 드리고 몇 가지는 이런 식으로 일일이 만들어 볼 건데, 그 과정을 거치면서 수많은 연구자들이 어떻게 효율적인 아키텍처를 구성했는가? 그리고 요즘에는 어떤 방식으로 사용하고 있는가까지 공부해 보면 좋을 것이다.

## 오늘 같이 해본 것들은?
1. 딥러닝을 학습시키는 단계를 복습!
2. 우리가 만들 CNN 구조 확인!
3. 필요한 함수들의 사용 방법을 확인
4. MNIST에 CNN 적용 코드를 함께 작성

## What's Next?
- 모델을 조금 더 깊게 만들어보고 차이를 확인하기
- Visdom이라는 시각화 도구도 살펴볼 것